In [1]:
## lets put all together

# import Pytorch and matplotlib

import torch, matplotlib.pyplot as plt
import torch.nn as nn

torch.__version__

'2.7.1'

In [2]:
# create device-agnostic code

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

In [3]:
# 6. Data 
# y = w*x + b
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(1)
y = weight * X + bias


In [4]:
# split data
train_split = int(0.8 *len(X))
X_train, y_train = X[:train_split], y[:train_split]

X_test, y_test = X[train_split:], y[train_split:]



In [5]:
## Plot the data
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})
  plt.show()


In [6]:
# plot_predictions(X_train, y_train, X_test, y_test)

In [ ]:
# building a Pytorch linear model
# this model is basically the same but more common
class LinearRegressionModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        ## use nn.Linear() for creating the model parms
        self.linear_layer = nn.Linear(in_features=1,
                                       out_features=1)
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)
    
# set the manual seed
torch.manual_seed(42)

model_1 = LinearRegressionModelV2()
model_1, model_1.state_dict()



(LinearRegressionModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [ ]:
# check the model current device
next(model_1.parameters()).device

model_1.to(device)   ## but only cpu is available for me

LinearRegressionModelV2(
  (linear_layer): Linear(in_features=1, out_features=1, bias=True)
)

#### Training

 we need:
 * Loss function
 * Optimizer
 * Training loop
 * Testing loop


In [12]:
loss_fn = nn.L1Loss()

# setup our optimizer
optimizer = torch.optim.SGD(model_1.parameters(), lr=0.1)

In [16]:
### Lets write a training loop
torch.manual_seed(42)

epochs = 201

for epoch in range(epochs):
    model_1.train()

    # 1. forward pass 
    y_pred = model_1(X_train)

    # 2. calculate the loss
    loss = loss_fn(y_pred, y_train)

    # 3. optimizer zero grad
    optimizer.zero_grad()

    # perform backpropagation
    loss.backward()

    # 4. optimizer step
    optimizer.step()

    ### Testing 

    model_1.eval()
    with torch.inference_mode():
        test_pred = model_1(X_test)
        test_loss = loss_fn(test_pred, y_test)
    if epoch % 10 ==0:
       
       print(f'Epoch: {epoch}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}')







Epoch: 0, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 10, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 20, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 30, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 40, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 50, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 60, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 70, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 80, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 90, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 100, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 110, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 120, Train Loss: 0.05761389806866646, Test Loss: 0.11862673610448837
Epoch: 130, Train Loss:

In [19]:
model_1.state_dict(), (weight, bias)

(OrderedDict([('linear_layer.weight', tensor([[0.5779]])),
              ('linear_layer.bias', tensor([0.2900]))]),
 (0.7, 0.3))

In [ ]:
# Making and evaluating predictions

# turn the model into evaluation mode

model_1.eval()

# make preds on the test data

with torch.inference_mode():
    test_pred = model_1(X_test)
    test_loss = loss_fn(test_pred, y_test)


In [ ]:
### Saving and loading

